# Naas - Demo Ontologies & Assistants

## Input

### Import libraries

In [1]:
import naas_data_product
from abi.naaspython import NaasPython
naaspython = NaasPython()
import naas_python
import pydash as _
import json
from datetime import datetime
from urllib.parse import quote
from algoliasearch.search.client import SearchClient

✅ utils file '/home/ftp/abi/utils/main.ipynb' successfully loaded.


### Setup variables

In [2]:
# Input
api_key = naas_python.utils.domains_base.authorization.NaasSpaceAuthenticatorAdapter().jwt_token()
workspace_id = naas_python.secret.get("WORKSPACE_DEMO_ID").value
algolia_app_id = naas_python.secret.get("ALGOLIA_APP_ID").value
algolia_api_key = naas_python.secret.get("ALGOLIA_API_KEY").value
data_algolia = []

# Outputs
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "demo")
index_name = "workspace-search"

## Model

### Get Ontologies

In [3]:
ontologies = naaspython.connect(api_key).ontology.get(workspace_id).get("ontologies")
print("Ontologies:", len(ontologies))

for x in ontologies:
    label = x.get("label")
    if not "Old" in label:
        uid = x.get("id")
        label = x.get("label")
        download_url = x.get("download_url")
        description = x.get("description")
        logo_url = x.get("logo_url")
        level = x.get("level")
        # Get tool
        tool = "Naas"
        if "-" in label:
            tool = label.split("-")[0].strip()
        # Get ranking & author
        author = "Naas Core Team"
        author_url = ""
        ranking = 2
        if level == "TOP_LEVEL":
            ranking = 5
            author = "BFO Team (Barry Smith)"
        elif level == "MID_LEVEL":
            ranking = 4
            author = "CCO Team"
        elif level == "DOMAIN":
            ranking = 2
            
        # Create Search data JSON
        data_algolia.append(
            {
              "tool": tool,
              "notebook": label,
              "tags": [
                  "#ontologies",
                  f"#{level.replace('_', '').lower()}"
              ],
              "author": author,
              "author_url": author_url,
              "updated_at": datetime.now().strftime("%Y-%m-%d"),
              "created_at": datetime.now().strftime("%Y-%m-%d"),
              "description": description,
              "open_in_lab": "",
              "open_in_chat": "",
              "notebook_url": download_url,
              "image_url": logo_url,
              "ranking": ranking,
              "objectID": uid
            }
        )

Ontologies: 29


### Get Plugins

In [4]:
workspace_plugins = naaspython.connect(api_key, workspace_id).plugin.get().get("workspace_plugins")
print("Workspace Plugins:", len(workspace_plugins))

for workspace_plugin in workspace_plugins:
    # Init
    uid = workspace_plugin.get("id")
    plugin = json.loads(workspace_plugin.get("payload"))
    
    # Map fields
    ranking = 3
    tool = "Naas Demo"
    label = plugin.get("name")
    short_name = label.lower().replace(' ', '')
    image_url = plugin.get("avatar")
    description = plugin.get("description")
    model = plugin.get("model")
    id = plugin.get("id")
    temperature = plugin.get("temperature")
    author = "Naas Core Team"
    author_url = ""
    
    # Save plugin json
    file_path = os.path.join(output_dir, "assistants", f"{short_name}.json")
    with open(file_path, 'w') as json_file:
        json.dump(plugin, json_file, indent=4)
    
    # Create Search data JSON
    data_algolia.append(
        {
          "tool": tool,
          "notebook": label,
          "tags": [
              "#assistants",
              f"#{short_name.split('.')[-1]}",
              f"#{model}",
              f"#avatar",
              f"#temperature:{temperature}",
          ],
          "author": author,
          "author_url": author_url,
          "updated_at": datetime.now().strftime("%Y-%m-%d"),
          "created_at": datetime.now().strftime("%Y-%m-%d"),
          "description": description,
          "open_in_lab": "",
          "open_in_chat": "",
          "notebook_url": quote(f"https://raw.githubusercontent.com/jupyter-naas/abi/refs/heads/main/outputs/demo/assistants/{short_name}.json", safe=":/#"),
          "image_url": image_url,
          "ranking": ranking,
          "objectID": uid
        }
    )

Workspace Plugins: 37


## Output

### Udpate Data Algolia

In [5]:
# Initialize the Algolia client
_client = SearchClient(algolia_app_id, algolia_api_key)

# Update Algolia
print("Data to update:", len(data_algolia))
for d in data_algolia:
    # Add a new record to your Algolia index
    response = await _client.save_object(
        index_name=index_name,
        body=d,
    )
    print(response)

Data to update: 66
created_at='2024-10-23T17:25:23.373Z' task_id=266734086001 object_id='c69f8602-8472-4174-9e89-0d7053591a41'
created_at='2024-10-23T17:25:23.518Z' task_id=266734090001 object_id='b22adec7-923f-4520-83fb-d7b483490df3'
created_at='2024-10-23T17:25:23.663Z' task_id=266734093001 object_id='fe262dab-b7b5-43d0-bd7e-91401a2d535b'
created_at='2024-10-23T17:25:23.809Z' task_id=266734097001 object_id='7056c1ec-ca21-42b6-99ad-fb5defd238f2'
created_at='2024-10-23T17:25:23.955Z' task_id=266734100001 object_id='675089d4-1a8d-4f6f-a0ba-7fef24f7b6ad'
created_at='2024-10-23T17:25:24.100Z' task_id=266734104001 object_id='9287ce2c-0086-41f6-988e-eeee9941d5da'
created_at='2024-10-23T17:25:24.245Z' task_id=266734108001 object_id='e011e7fc-1e88-4a16-b994-70475ddb3bfc'
created_at='2024-10-23T17:25:24.391Z' task_id=266734111001 object_id='ac41c275-ba7c-460c-a189-0d3acfeb4a05'
created_at='2024-10-23T17:25:24.536Z' task_id=266734116001 object_id='97da064e-8a58-4c18-80be-70f7437b27c0'
created_a